## CoDA methods
Because traditional tests will lead to spurious results, it is recommended to transform or normalize your post-ASV clustering data before computing alpha and beta (distance calculation) diversity, differential abundance and ordinations (unsupervised clustering).

Outside of QIIME2 using Python 3

The first thing you will need to do is export your feature table from QIIME2 to a .tsv file

In your terminal,
</br>
1. Activate your QIIME2 environment (whichever it is)
    * `conda activate qiime2.2020.11`
2. Move to your working directory with your files
    * cd FILEPATH
3. Clone from github
    * `git clone https://github.com/dianahaider/normalization_pipeline`
4. Move to the cloned directory and run this code to make the scripts executable (it gives permission to the files to be executable)
    * `chmod a+x ./*`
5. Run ~/normalization-pipeline/export-asv-results.sh
    * You might get an error if you use a mac about the shell name, just make sure your QIIME2 environment is active
    * If you use LINUX, and you get an error, make sure the qiime2 version in the file is the same as you use in your computer (I use .2020.11, just change it to whichever you are using).

This should outputs two files in a new directory `exported_table`
* feature-table.biom.tsv
* feature-table.biom

Now, go back to the cloned directory from github, and run `jupyter notebook` and open this file in jupyter notebook and follow along the code below

In [164]:
#import the relevant packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA
from skbio.diversity import alpha_diversity
from skbio.stats.distance import permanova
from skbio import DistanceMatrix
from scipy.spatial.distance import cdist
from skbio.stats.composition import clr
from skbio.stats.composition import alr
from skbio.stats.composition import ilr
from skbio.diversity.alpha import chao1

In [140]:
#First you have to import your tsv table to the notebook
feature_table_sparse = pd.read_csv('~/normalization_pipeline/test/exported_table/feature-table.biom.tsv', sep='\t',skiprows=1,index_col=0)

In [141]:
#Preview your table to make sure it was correctly exported from QIIME and imported to jup
feature_table_sparse.head()

,L1S105,L1S140,L1S208,L1S257,L1S281,L1S57,L1S76,L1S8,L2S155,L2S175,...,L4S63,L5S104,L5S155,L5S174,L5S203,L5S222,L5S240,L6S20,L6S68,L6S93
#OTU ID,,,,,,,,,,,,,,,,,,,,,
4b5eeb300368260019c1fbc7a3c718fc,2175.0,0.0,0.0,0.0,0.0,2806.0,3309.0,2595.0,10.0,10.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fe30ff0f71a38a39cf1717ec2be3a2fc,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,160.0,0.0,0.0,0.0,0.0,0.0,374.0,3323.0,1723.0,1341.0
d29fe3c70564fc0f69f2c03e0d1e5561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,353.0,779.0,...,417.0,216.0,140.0,107.0,215.0,148.0,117.0,215.0,500.0,465.0
868528ca947bc57b69ffdf83e6b73bae,0.0,2249.0,2107.0,1177.0,1722.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,10.0,0.0,0.0,9.0
154709e160e8cada6bfb21115acc80f5,803.0,1174.0,694.0,406.0,242.0,1081.0,930.0,1623.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Sparse feature table
If your table contains 0s (most likely it does), you need to add a pseudo-count to it (a small value). There is <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5682008/pdf/fmicb-08-02114.pdf" target="_blank">litterature</a> that suggests other methods, but this one is widely used and accepted. If you want to dive into it, there are 3 types of 0s <br>
* Real 0s (structural 0s)
    <br> i.e taxa not supposed to be present, and not detected
* False 0s (either outlier 0, or sampling 0s)
    1. outlier 0 
    <br> i.e taxa not sampled for unknown reasons
    2. sampling 0
    <br> i.e taxa not sampled because of sampling depth

In [137]:
#Add a 0.1 pseudo count to all 0s to compute log
feature_table=feature_table_sparse.mask(feature_table==0).fillna(0.1)

In [71]:
#The head function allows us to look at the 5 first rows
feature_table_sparse.head()

,L1S105,L1S140,L1S208,L1S257,L1S281,L1S57,L1S76,L1S8,L2S155,L2S175,...,L4S63,L5S104,L5S155,L5S174,L5S203,L5S222,L5S240,L6S20,L6S68,L6S93
#OTU ID,,,,,,,,,,,,,,,,,,,,,
4b5eeb300368260019c1fbc7a3c718fc,2175.0,0.0,0.0,0.0,0.0,2806.0,3309.0,2595.0,10.0,10.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fe30ff0f71a38a39cf1717ec2be3a2fc,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,160.0,0.0,0.0,0.0,0.0,0.0,374.0,3323.0,1723.0,1341.0
d29fe3c70564fc0f69f2c03e0d1e5561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,353.0,779.0,...,417.0,216.0,140.0,107.0,215.0,148.0,117.0,215.0,500.0,465.0
868528ca947bc57b69ffdf83e6b73bae,0.0,2249.0,2107.0,1177.0,1722.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,10.0,0.0,0.0,9.0
154709e160e8cada6bfb21115acc80f5,803.0,1174.0,694.0,406.0,242.0,1081.0,930.0,1623.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
#clr transformation is applied to the dataframe (with the row and column names), but it
#ouputs an array so we will store the result, and reconstruct the dataframe
#We could have used alr or ilr too
clr_transformed_array = clr(feature_table)

In [47]:
#storing the sample and asv names from the original dataframe
samples = feature_table.columns
asvs = feature_table.index

In [48]:
#Creating the dataframe with the clr transformed data, and assigning the sample names
clr_transformed = pd.DataFrame(clr_transformed_array, columns=samples)

In [177]:
#Assigning the asv names
clr_transformed['asvs'] = asvs
clr_transformed = clr_transformed.set_index('asvs')

In [121]:
clr_transformed.head()

,L1S105,L1S140,L1S208,L1S257,L1S281,L1S57,L1S76,L1S8,L2S155,L2S175,...,L4S63,L5S104,L5S155,L5S174,L5S203,L5S222,L5S240,L6S20,L6S68,L6S93
asvs,,,,,,,,,,,,,,,,,,,,,
4b5eeb300368260019c1fbc7a3c718fc,7.391545,-2.595824,-2.595824,-2.595824,-2.595824,7.646276,7.811162,7.568103,2.009346,2.009346,...,-2.595824,2.009346,-2.595824,-2.595824,-2.595824,-2.595824,-2.595824,-2.595824,-2.595824,-2.595824
fe30ff0f71a38a39cf1717ec2be3a2fc,0.492488,-3.419535,-3.419535,-3.419535,-3.419535,-3.419535,-3.419535,-3.419535,-3.419535,-3.419535,...,3.958224,-3.419535,-3.419535,-3.419535,-3.419535,-3.419535,4.807306,6.991673,6.334872,6.084221
d29fe3c70564fc0f69f2c03e0d1e5561,-5.557092,-5.557092,-5.557092,-5.557092,-5.557092,-5.557092,-5.557092,-5.557092,2.611961,3.403504,...,2.778579,2.120771,1.687135,1.418322,2.116131,1.742705,1.507667,2.116131,2.960101,2.887530
868528ca947bc57b69ffdf83e6b73bae,-2.072053,7.948773,7.883553,7.301256,7.681774,-2.072053,-2.072053,-2.072053,-2.072053,-2.072053,...,-2.072053,-2.072053,-2.072053,-2.072053,1.839970,-2.072053,2.533117,-2.072053,-2.072053,2.427757
154709e160e8cada6bfb21115acc80f5,6.065054,6.444871,5.919171,5.383052,4.865637,6.362341,6.211884,6.768731,-2.925886,-2.925886,...,-2.925886,-2.925886,1.573924,-2.925886,-2.925886,-2.925886,-2.925886,-2.925886,-2.925886,-2.925886


### Alpha diversity ** Not completed
The CoDA method for alpha diversity is to compute CHAO1 (richness) or Shannon (evenness). <br>
The CHAO1 index input is non-rarefied/non-transformed data (aka raw counts) and the Shannon diversity index takes normalized/rarefied/transformed

In [77]:
#This will calculate the number of samples this asv was seen in
obs_otus = alpha_diversity('observed_otus', feature_table_sparse, asvs)
obs_otus

#OTU ID
4b5eeb300368260019c1fbc7a3c718fc    13
fe30ff0f71a38a39cf1717ec2be3a2fc    16
d29fe3c70564fc0f69f2c03e0d1e5561    25
868528ca947bc57b69ffdf83e6b73bae    10
154709e160e8cada6bfb21115acc80f5    13
                                    ..
a6b6f29a1196cacfc392e3d71f55e2a2     1
0e5df3d01cc073e3c9674c2534169f03     1
06845c67bc4203081a981200f33e87eb     1
98d250a339a635f20e26397dafc6ced3     1
1830c14ead81ad012f1db0e12f8ab6a4     1
Length: 770, dtype: int64

In [75]:
chao1 = samples.plot_metadata(vaxis="chao1", haxis="geo_loc_name", return_chart=True)

In [76]:
adiv_faith_pd

0      13.0
1      16.0
2      25.0
3      10.0
4      13.0
       ... 
765     1.0
766     1.0
767     1.0
768     1.0
769     1.0
Length: 770, dtype: float64

### Beta diversity
The CoDA method for beta diversity calculations is the Aitchison distance, which is simply the Euclian distance between samples after clr transformation

In [83]:
#You need to use the transpose of the table because our columns are samples, and cdist computes the distance
#between pairs of rows, not pairs of columns
dist = cdist(clr_transformed.T, clr_transformed.T, 'euclid')

In [85]:
#Just as we previously did with clr, we need to reconstruct the table after applying cdist
distance_matrix = pd.DataFrame(dist, columns=samples)

In [89]:
distance_matrix['samples'] = samples
distance_matrix = distance_matrix.set_index('samples')

In [168]:
distance_matrix

,L1S105,L1S140,L1S208,L1S257,L1S281,L1S57,L1S76,L1S8,L2S155,L2S175,...,L4S63,L5S104,L5S155,L5S174,L5S203,L5S222,L5S240,L6S20,L6S68,L6S93
samples,,,,,,,,,,,,,,,,,,,,,
L1S105,0.000000,50.660476,53.009714,53.688080,53.458794,39.266192,34.249881,40.786794,68.202433,68.152313,...,89.547740,54.552149,52.926555,55.141802,55.760395,55.467756,53.600612,60.315564,59.753785,62.806810
L1S140,50.660476,0.000000,48.609390,46.530973,45.727517,55.013435,50.059470,45.652815,69.188427,69.567043,...,90.416556,56.003679,53.537745,55.759361,55.671413,56.081726,53.977640,60.974521,60.438490,63.836860
L1S208,53.009714,48.609390,0.000000,34.792111,35.981231,54.630893,50.566306,54.161753,73.835605,73.744846,...,94.433156,60.286656,57.969353,59.986803,59.928604,60.457243,58.880162,64.833009,64.421187,67.502259
L1S257,53.688080,46.530973,34.792111,0.000000,29.785402,55.720405,50.079869,54.086832,72.480833,72.447081,...,93.380999,59.262530,56.963472,59.007058,58.949004,59.400300,57.392055,63.917520,63.452446,66.708979
L1S281,53.458794,45.727517,35.981231,29.785402,0.000000,53.204023,47.877624,54.981851,72.469797,72.510591,...,93.494698,58.768556,56.549391,58.535779,58.469875,58.842936,56.862781,63.501542,62.989385,66.293881
L1S57,39.266192,55.013435,54.630893,55.720405,53.204023,0.000000,30.930870,46.487716,71.582245,72.055573,...,93.659673,58.206829,56.606645,58.747458,59.305199,59.169595,57.879971,64.218362,63.710016,67.102688
L1S76,34.249881,50.059470,50.566306,50.079869,47.877624,30.930870,0.000000,42.015113,68.701584,68.860753,...,91.219470,54.199856,52.587557,54.828556,55.450644,55.156362,53.878986,60.704014,60.103128,63.736901
L1S8,40.786794,45.652815,54.161753,54.086832,54.981851,46.487716,42.015113,0.000000,63.958098,63.708223,...,87.364866,47.142601,45.180677,47.878171,48.478772,48.320461,46.724653,54.222330,53.610409,57.663009
L2S155,68.202433,69.188427,73.835605,72.480833,72.469797,71.582245,68.701584,63.958098,0.000000,48.545925,...,74.196641,54.607383,55.176531,56.785390,55.066895,56.366996,55.675101,61.066463,59.391549,62.615739


In [151]:
#Plot a covariance matrix to visualize the distance between each pairs of samples
fig = px.imshow(distance_matrix)
fig.show()

In [115]:
#Let's import the metadata to make sense of the PCA
#skip rows is now in brackets, meaning we remove the row 1, not the first row (python counts 0,1,2...)
mtda = pd.read_csv('~/normalization_pipeline/test/sample-metadata.tsv', sep='\t',skiprows=[1])

In [116]:
mtda.head()

,sample-id,barcode-sequence,body-site,year,month,day,subject,reported-antibiotic-usage,days-since-experiment-start
0,L1S8,AGCTGACTAGTC,gut,2008,10,28,subject-1,Yes,0
1,L1S57,ACACACTATGGC,gut,2009,1,20,subject-1,No,84
2,L1S76,ACTACGTGTGGT,gut,2009,2,17,subject-1,No,112
3,L1S105,AGTGCGATGCGT,gut,2009,3,17,subject-1,No,140
4,L2S155,ACGATGCGACCA,left palm,2009,1,20,subject-1,No,84


In [176]:
#In order to compute the permanova test, the distance_matrix has to be reformatted
dm = DistanceMatrix(distance_matrix)

A permanova test statistically determines if two groups (their centre in geometrical space) are different. Are the samples different between body sites? <a href="https://onlinelibrary.wiley.com/doi/pdf/10.1002/9781118445112.stat07841" target="_blank">More readings</a>

In [175]:
#perMANOVA tests the association of the microbiome composition with any of the covariate of
#interest, 'body-site' is the column name of the covariate from your metadata
permanova(dm, grouping=mtda['body-site'])

method name               PERMANOVA
test statistic name        pseudo-F
sample size                      34
number of groups                  4
test statistic              1.72268
p-value                       0.007
number of permutations          999
Name: PERMANOVA results, dtype: object

### Make a PCA plot to visualize your data

In [154]:
#Relationship between samples with the distance matrix
pca = PCA(n_components=2)
components = pca.fit_transform(distance_matrix)

fig = px.scatter(components, x=0, y=1, color=mtda['body-site'])
fig.show()

In [135]:
#Relationship between samples with the clr transformed data
pca = PCA(n_components=2)
components = pca.fit_transform(clr_transformed.T)

fig = px.scatter(components, x=0, y=1, color=mtda['body-site'])
fig.show()

In [142]:
#Here is the PCA with the raw data
pca = PCA(n_components=2)
components = pca.fit_transform(feature_table_sparse.T)

fig = px.scatter(components, x=0, y=1,color=mtda['body-site'])
fig.show()

In [134]:
#Relationship between samples in 3d
pca = PCA(n_components=3)
components = pca.fit_transform(clr_transformed.T)

fig = px.scatter_3d(components, x=0, y=1, z=2, color=mtda['body-site'])
fig.show()

In [143]:
#Relationship between samples in 3d
pca = PCA(n_components=3)
components = pca.fit_transform(feature_table_sparse.T)

fig = px.scatter_3d(components, x=0, y=1, z=2, color=mtda['body-site'])
fig.show()